# Single Class TB

In [14]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3999)])
 

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

ValueError: Cannot set memory growth on device when virtual devices configured

In [ ]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))

Load and prepare data

In [15]:
import cv2 as cv
import pickle
import random
import numpy as np
 

In [24]:
raw_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/tb1-256-grayscale", "rb" ) ) # array of tuples( [x,y], 0|1 )
random.shuffle(raw_data)


fraction_train = 0.8
train_size = int(len(raw_data) * fraction_train)

x_train = []
y_train = []

for v in raw_data[0: train_size] :
    x_train.append(v[0])
    y_train.append(v[1])

x_train = np.array(x_train) 
y_train = np.array(y_train)    

x_test = []
y_test = []


for v in raw_data[train_size:]:
    x_test.append(v[0])
    y_test.append(v[1])
    

x_test = np.array(x_test) 
y_test = np.array(y_test)  


x_train, x_test = x_train / 255.0, x_test / 255.0



# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

print("train size: {}, test size: {}".format(len(x_train), len(x_test)))


train size: 529, test size: 133


Use `tf.data` to batch and shuffle the dataset:

In [25]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(100).batch(16)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(16)

In [26]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(64, activation='relu')
    self.d2 = Dense(2)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

Choose an optimizer and loss function for training: 

In [27]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

Select metrics to measure the loss and the accuracy of the model. These metrics accumulate the values over epochs and then print the overall result.

In [28]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use `tf.GradientTape` to train the model:

In [29]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

Test the model:

In [30]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [31]:
EPOCHS = 30

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    'Epoch {}, '.format(epoch + 1),
    'Loss: {}, '.format(train_loss.result()),
    'Accuracy: {}, '.format(train_accuracy.result() * 100),
    'Test Loss: {}, '.format(test_loss.result()),
    'Test Accuracy: {}'.format(test_accuracy.result() * 100)
  )

Epoch 1,  Loss: 19.57366371154785,  Accuracy: 51.41776657104492,  Test Loss: 0.6197091937065125,  Test Accuracy: 69.17292785644531
Epoch 2,  Loss: 0.5636592507362366,  Accuracy: 72.77882385253906,  Test Loss: 0.4881576895713806,  Test Accuracy: 77.44361114501953
Epoch 3,  Loss: 0.43316781520843506,  Accuracy: 83.93194580078125,  Test Loss: 0.6272428035736084,  Test Accuracy: 77.44361114501953
Epoch 4,  Loss: 0.46692344546318054,  Accuracy: 79.77315521240234,  Test Loss: 0.4820440113544464,  Test Accuracy: 78.19548797607422
Epoch 5,  Loss: 0.3517055809497833,  Accuracy: 86.57845306396484,  Test Loss: 0.5050029754638672,  Test Accuracy: 80.45112609863281
Epoch 6,  Loss: 0.3435397148132324,  Accuracy: 86.20037841796875,  Test Loss: 0.5633062124252319,  Test Accuracy: 79.69924926757812
Epoch 7,  Loss: 0.33853816986083984,  Accuracy: 87.52362823486328,  Test Loss: 0.5948857665061951,  Test Accuracy: 77.44361114501953
Epoch 8,  Loss: 0.34219327569007874,  Accuracy: 86.38941955566406,  Test L